En el apartado "Loading Data" de esta URL:

https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python

Se explica cómo se cargan una serie de datos: 

1. Utiliza esa misma forma para cargar los datos.
2. Limpia los datos si es necesario
3. Grafica los datos que creas necesarios
4. Utiliza los métodos de clasificación vistos hasta ahora para clasificar el target de los datos, ¿cuál da mejores resultados? 
5. Intenta superarte en el score cambiando las features de los algoritmos.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

import plotly.offline as py
import plotly.graph_objs as go

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold

In [2]:
cancer = datasets.load_breast_cancer()

In [3]:
print("Features: ", cancer.feature_names)

print("Labels: ", cancer.target_names)

Features:  ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
Labels:  ['malignant' 'benign']


In [4]:
# print the cancer labels (0:malignant, 1:benign)
# print(cancer.target)

In [5]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [6]:
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [7]:
cancer_df = pd.DataFrame(data= np.c_[cancer.data, cancer.target],
                     columns= list(cancer.feature_names) + ['target'] )

In [8]:
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

**2. Limpia los datos si es necesario**

Estos datos ya han sido tratados y preparados por `sklearn`, no se observan datos nulos, ni duplicados, todos los datos son numéricos y están bien nombrados. No creo que sea necesaria más limpieza.

- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html
- https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset

**3. Grafica los datos que creas necesarios**

In [9]:
mean_col = []
error_col = []
worst_col = []
area_col = []

for col in cancer_df.columns.tolist():
    if "area" in col:
        area_col.append(col)
    else:
        if col[:4] == "mean":
            mean_col.append(col)
        elif col[-5:] == "error":
            error_col.append(col)
        elif col[:5] == "worst":
            worst_col.append(col)

In [10]:
print(f"Divido las features en diferentes listas para hacer más fácil el estudio de estas:\n\n\
mean_col:\n{mean_col}\n\nerror_col:\n{error_col}\n\nworst_col:\n{worst_col}\n\narea_col:\n{area_col}")

Divido las features en diferentes listas para hacer más fácil el estudio de estas:

mean_col:
['mean radius', 'mean texture', 'mean perimeter', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension']

error_col:
['radius error', 'texture error', 'perimeter error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error']

worst_col:
['worst radius', 'worst texture', 'worst perimeter', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']

area_col:
['mean area', 'area error', 'worst area']


In [11]:
y = cancer_df["target"]
X = cancer_df.drop("target", 1)

In [12]:
print(f"There are {len(y[y == 0])} entries for malignant cancer and {len(y[y == 1])} for benign")

There are 212 entries for malignant cancer and 357 for benign


In [13]:
diagnosis = cancer_df['target'].value_counts()
diagnosis_size = diagnosis.values

colors = ['lavender', 'lightgreen']

trace = go.Pie(labels = ["Malignant", "Benign"],
              values = diagnosis_size,
               marker = dict(colors = colors),
               name = 'Breast Cancer',
               hole = 0.3
              )
df = [trace]
layout = go.Layout(title = 'Distribution of Patients')


fig = go.Figure(data = df, layout = layout)
fig.write_html("pie.html")

py.iplot(fig)

/Users/iudh/opt/anaconda3/envs/clase/lib/python3.8/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



In [14]:
layout = go.Layout(title = 'Statistics for the area features')
fig = go.Figure(layout = layout)

colors = ["lightseagreen", "goldenrod",  'aqua']

for i, elem in enumerate(area_col): 
    fig.add_trace(go.Box(y=cancer_df[elem], name=elem,
                marker_color = colors[i]))

fig.write_html("box.html")
fig.show()

In [15]:
layout = go.Layout(title = 'Features correlation matrix')
fig = go.Figure(data=go.Heatmap(x=cancer_df.corr().columns.tolist(),
                                y=cancer_df.corr().columns.tolist(),
                                z = np.array(cancer_df.corr()),
                                colorscale = 'Viridis'),
                                layout = layout)
fig.write_html("heatmap.html")
fig.show()

Los gráficos estánn guardados en esta misma carpeta en formato `html`.

Os dejo por aquí un análisis de las features hecha con `matplotlib` que merece la pena echar un ojo:

https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-multicollinear-py

**4. Utiliza los métodos de clasificación vistos hasta ahora para clasificar el target de los datos, ¿cuál da mejores resultados?** 

**5. Intenta superarte en el score cambiando las features de los algoritmos.**

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
pipe = Pipeline(steps=[('classifier', LogisticRegression())])


logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.logspace(0, 4, 10)
    }

kneighbors_params = {
    'classifier': [KNeighborsClassifier()],
    'classifier__n_neighbors': [3,5,11,19],
    'classifier__weights': ["uniform", "distance"],
    'classifier__metric': ["euclidean", "manhattan"]
    }

svm_params = {
    'classifier': [SVC()],
    'classifier__kernel':('linear', 'rbf', 'sigmoid'), 
    'classifier__C':[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
    'classifier__coef0': [-10.,-1., 0., 0.1, 0.5, 1, 10, 100],
    'classifier__gamma': ('scale', 'auto')
    }


# Create space of candidate learning algorithms and their hyperparameters
search_space = [
    logistic_params,
    kneighbors_params,
    svm_params
    ]

In [18]:
%%time
cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, cv = cv, verbose=4, n_jobs=-1)

# Fit grid search
best_model = clf.fit(X_train, y_train)
# View best model
separator = "\n############################\n"
print(separator)
print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
filename = 'finished_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 420 candidates, totalling 4200 fits


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 938 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 1280 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 1722 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 2260 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done 2898 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 3541 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 4105 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 4185 out of 4200 | elapsed:  2.2min remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 4200 out of 4200 | elapsed:  2.2min finished



############################

best estimator: SVC(C=0.5, coef0=-10.0, kernel='linear')

############################

clf.best_params_ {'classifier': SVC(C=0.5, coef0=-10.0, kernel='linear'), 'classifier__C': 0.5, 'classifier__coef0': -10.0, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}

############################

clf.best_score 0.9540579710144929
CPU times: user 7.79 s, sys: 543 ms, total: 8.33 s
Wall time: 2min 14s


In [19]:
with open("finished_model.sav","rb") as f:
    loaded_model = pickle.load(f)

In [20]:
best_model.score(X_test, y_test) * 100

96.49122807017544